### notes

### L_mC - 110117

I'd be keen to explore:
* relationships between the fields and data. 
* highlighting high bounce zones, 
* dead spots, 
* zones, 
* relationship between speed and xyz. 
* increased / decreased bounce over time of day etc.

x = depth
y = horizon
z = height

### import libraries

In [155]:
import notebook
import pandas as pd
import numpy as np
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D as p3
from matplotlib import pyplot as plt
from matplotlib import colors, ticker, cm
import matplotlib.patches as mpatches
import seaborn as sns
import notebook
from scipy.stats import kendalltau
from bokeh.io import output_notebook, show
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool, BoxSelectTool, BoxZoomTool, LassoSelectTool, ResetTool
from mpl_toolkits.mplot3d import Axes3D
import plotly
plotly.tools.set_credentials_file(username='linus.austen.mcmanamey', api_key='3tqgEzyZAGTIeFY5sQ1T')
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.graph_objs import Surface, Layout, Scene
import gr
from os import environ
environ['MPLBACKEND'] = 'module://gr.matplotlib.backend_gr'
import numpy as np
%matplotlib inline

ImportError: No module named GR

In [135]:
output_notebook()

Loading BokehJS ...

### The wickets
Two sets of wickets shall be pitched opposite and parallel to each other at a distance of 22 yards/20.12 m between the centres of the two middle stumps. Each set shall be 9 in/22.86 cm wide and shall consist of three wooden stumps with two wooden bails on top. See Appendix A.

The tops of the stumps shall be 28 in/71.1 cm above the playing surface and shall be dome shaped except for the bail grooves. The portion of a stump above the playing surface shall be cylindrical apart from the domed top, with circular section of diameter not less than 1⅜ in/3.49 cm nor more than 1½ in/3.81 cm. See Appendix A.

https://www.lords.org/mcc/laws-of-cricket/laws/law-8-the-wickets/

In [136]:
#### Global Wicket Variables
# measurement in meters
LENGTH_OF_PITCH = 20.12
WIDTH_OF_PITCH = 3.66
WIDTH_OF_STUMPS = 0.2286
HEIGHT_OF_STUMPS = 0.711
HEIGHT_OF_BAILS = 0.0349
STUMPS = HEIGHT_OF_STUMPS + HEIGHT_OF_BAILS
#### measurement for graphing 
PITCH_WIDTH = 366
PITCH_LENGTH = 2012
PITCH_SCALE = 1

In [137]:
#### matplotlib graph size
plt.rcParams['figure.figsize']=(50,10)

### read in csv file

In [138]:
df = pd.read_csv('day5.csv', parse_dates=True)

### strip out leading spaces

In [139]:
df = df.rename(columns=lambda x: x.strip())

### convert TimeBowled to time object

In [140]:
df.TimeBowled = pd.to_datetime(df.TimeBowled)

###  set TimeBowled to index

In [141]:
df = df.set_index(pd.DatetimeIndex(df['TimeBowled']),inplace=False)

### strip out hour for time sequencing

In [142]:
df['hour'] = df.index.hour

### cast variables as floats

In [143]:
df.releaseX = df.releaseX.astype('float64')
df.releaseSpeed = df.releaseSpeed.astype('float64')
df.dropAngle = df.dropAngle.astype('float64')
df.bounceX = df.bounceX.astype('float64')
df.bounceSpeed = df.bounceSpeed.astype('float64')
df.bounceAngle = df.bounceAngle.astype('float64')

### convert universal co-ordinates to seperate bowling ends

In [144]:
def releaseXMod(row):
    if(row['Over'] == 0):
          rValue = row['releaseX'] - (LENGTH_OF_PITCH/2)
    if(row['Over'] % 2 != 0):
        rValue = -(row['releaseX'] - (LENGTH_OF_PITCH/2))
    else:
        rValue = row['releaseX'] - (LENGTH_OF_PITCH/2)
    return(rValue)   

In [145]:
def bounceXMod(row):
    if(row['Over'] == 0):
          rValue = (row['bounceX'] - (LENGTH_OF_PITCH/2))
    if(row['Over'] % 2 != 0):
        rValue = -(row['bounceX'] - (LENGTH_OF_PITCH/2))
    else:
        rValue = (row['bounceX'] - (LENGTH_OF_PITCH/2))
    return(rValue)    

In [146]:
def stumpPosXMod(row):
    if(row['Over'] == 0):
          rValue = -(LENGTH_OF_PITCH/2)
    if(row['Over'] % 2 != 0):
        rValue = (LENGTH_OF_PITCH/2)
    else:
        rValue = -(LENGTH_OF_PITCH/2)
    return(rValue)

In [147]:
def bounceZMod(row):
    if(row['Over'] == 0):
          rValue = 0.0001
    if(row['Over'] % 2 != 0):
        rValue = 0.0001
    else:
        rValue = 0.0001
    return(rValue)

### apply functions to dataframe

In [148]:
df['bounceX'] = df.apply(bounceXMod, axis=1)
df['releaseX'] = df.apply(releaseXMod, axis=1)
df['stumpPosX'] = df.apply(stumpPosXMod, axis=1)
df['bounceZ'] = df.apply(bounceZMod, axis=1)

### remove null values

In [149]:
df = df[df.bounceY > -2]
df.Batsman.dropna(how='any', inplace=True)

## bokeh plots

In [150]:
width = PITCH_WIDTH / PITCH_SCALE
height = PITCH_LENGTH / PITCH_SCALE

In [151]:
source = ColumnDataSource(data=dict(Bowler=df.Bowler,Batsman=df.Batsman,speed=df.bounceSpeed,rspeed=df.releaseSpeed,))
hover = HoverTool(tooltips=[("Bowler", "@Bowler"),("Batsman","@Batsman"),("Bounce Speed km/h","@speed"),("Release Speed km/h","@rspeed")])

In [152]:
p = figure(plot_width=width, plot_height=height,tools=[hover,BoxZoomTool(),ResetTool()],title="Mouse over the dots")
p.circle(df.bounceY, df.bounceX, size=4, source=source)
show(p)


In [154]:
gr.inline('mov')
gr.plot(df.bounceX, df.bounceY)
#temp = sns.jointplot(df.bounceX, df.bounceY, kind="hex")
gr.show()

AttributeError: 'module' object has no attribute 'plot'

In [ ]:
plt.scatter(df.bounceX, df.bounceY, edgecolors='none')
plt.grid(True)
plt.show()

In [ ]:
ax = sns.kdeplot(df.bounceX, df.bounceY, cmap="Blues", shade=True, shade_lowest=False)

In [ ]:
#fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')
#ax.scatter(df.bounceX, df.bounceY)
#plt.show()

In [ ]:
trace1 = go.Scatter3d(x=df.releaseX, y=df.releaseY, z=df.releaseZ, mode='markers', 
                    marker=dict(size=4,line=dict(color='red', width=1), 
                    opacity=0
)
)
trace2 = go.Scatter3d(
    x=df.bounceX,
    y=df.bounceY,
    z=df.bounceZ,
    text=['Text A', 'Text B', 'Text C'],
    mode='markers',
    marker=dict(
        color='blue',
        size=4,
        symbol='circle',
        line=dict(
            color='green',
            width=1
        ),
        opacity=0.9
    )
)

trace3 = go.Scatter3d(
    x=df.stumpPosX,
    y=df.stumpPosY,
    z=df.stumpPosZ,
    mode='markers',
    marker=dict(
        color='blue',
        size=4,
        symbol='circle',
        line=dict(
            color='yellow',
            width=1
        ),
        opacity=0.9
    )
)
data = [trace1, trace2, trace3]
#layout = go.Layout(scene=Scene(aspectmode="data"),autosize=False, width=1000, height=500,margin=dict(l=0,r=0, b=0,t=0))
layout = go.Layout(scene=Scene(aspectmode="data"),margin=dict(l=80,r=80, b=0,t=0))
#layout = go.Layout(autosize=False, width=10, height=5, margin=go.Margin(l=50, r=50, b=100, t=100, pad=4, paper_bgcolor='#7f7f7f', plot_bgcolor='#c7c7c7'))




fig = go.Figure(data=data, layout=layout)
py.iplot(fig, filename='simple-3d-scatter')

In [ ]:
help('Scatter3d')

In [ ]:
import plotly.plotly as py
from plotly.grid_objs import Grid, Column

import time

column_1 = Column([0.5], 'x')
column_2 = Column([0.5], 'y')
column_3 = Column([1.5], 'x2')
column_4 = Column([1.5], 'y2')

grid = Grid([column_1, column_2, column_3, column_4])
py.grid_ops.upload(grid, 'ping_pong_grid'+str(time.time()), auto_open=False)df.dtypes

In [2]:
from sklearn.datasets import load_iris
from sklearn.decomposition import FactorAnalysis
iris = load_iris()
X, y = iris.data, iris.target
factor = FactorAnalysis(n_components=4, random_state=101).fit(X)


In [3]:
factor

FactorAnalysis(copy=True, iterated_power=3, max_iter=1000, n_components=4,
        noise_variance_init=None, random_state=101,
        svd_method='randomized', tol=0.01)

In [4]:
import mdp

ImportError: No module named mdp